https://www.nhs.uk/conditions/

In [4]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [5]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}


In [6]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
    try:
        res = requests.get('https://www.nhs.uk/conditions/', headers=headers_pc, proxies= randomproxy)
        # 将cookieJar数据转化为字典，s
        cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    except Exception as e:
        time.sleep(3)
        res = requests.get('https://www.nhs.uk/conditions/', headers=headers_pc, proxies= randomproxy)
        # 将cookieJar数据转化为字典，s
        cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict


In [7]:
def get_response(url, headers_pc, randomproxy, cookie_dict):
    try:
        r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        return r.text
    except Exception as e:
        time.sleep(3)
        print(e)

    return None

In [8]:
def health_articlehtml_download(headers_pc, randomproxy, cookie_dict, item):
    try:
        url = item['url']
        title = item['term'].strip().replace('/', ' ').replace('?', ' ').replace('/"', ' ')
        if os.path.exists(f"D:/Intern_Project/7_nhs/health_html/{title}.html"):
            pbar.update(1)    
            return None 
#         time.sleep(random.randint(0, 5))
        response = get_response(url, headers_pc, randomproxy, cookie_dict)
        
        with open(f"D:/Intern_Project/7_nhs/health_html/{title}.html", 'w+', encoding='utf-8') as f:
            f.write(response)

        pbar.update(1)
    
    except Exception as e:
        print(e)
    return None

In [9]:
def task_monitor(task_list, mt_pool, results):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
#                 results.extend(result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [ ]:
# 下载health_html
with open('D:/Intern_Project/7_nhs/health术语链接列表.json', 'r', encoding='utf-8') as f:
        url_list = json.load(f)

pbar = tqdm(total=len(url_list))

all_craw_tasks = []
results = []
headers_pc, randomproxy, cookie_dict = changeparams()
with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for item in url_list:
        tmp_task = mt_pool.submit(health_articlehtml_download, headers_pc, randomproxy, cookie_dict, item)
        all_craw_tasks.append(tmp_task)

    # 监测下载任务进展
    task_monitor(all_craw_tasks, mt_pool, results)

 18%|██████████████▍                                                                | 225/1226 [00:03<01:44,  9.55it/s]

In [10]:
# 每个item里面的term的url对应的是具体article的url列表
def medicineslist_html_download(headers_pc, randomproxy, cookie_dict, item, article_urllist):
    try:
        url = item['url']
        term = item['term'].strip().replace('/', ' ').replace('?', ' ').replace('/"', ' ')
        if os.path.exists(f"D:/Intern_Project/7_nhs/medicineslist_html/{term}.html"):
            return None
#         time.sleep(random.randint(0, 5))
        response = get_response(url, headers_pc, randomproxy, cookie_dict)
        
        with open(f"D:/Intern_Project/7_nhs/medicineslist_html/{term}.html", 'w+', encoding='utf-8') as f:
            f.write(response)
        
        selector = Selector(text=response)
        article_list = selector.xpath(
            "//li[@class='nhsuk-hub-key-links__list-item beta-hub-key-links__list-item']/a")
        
        for articleUrl in article_list:
            url = articleUrl.xpath('./@href').extract()[0]
            title = articleUrl.xpath('./text()').extract()[0].strip()
            # print(term)
            tmp = {
                'title': title,
                'url': url
            }
            article_urllist.append(tmp)
    except Exception as e:
        print(e)
    return article_urllist

In [8]:
# 下载medicines_article_html
# 先下载medicineslist_html，获取article_urllist
with open('D:/Intern_Project/7_nhs/medicines术语链接列表.json', 'r', encoding='utf-8') as f:
        url_list = json.load(f)

article_urllist = []

for item in url_list:
    results = medicineslist_html_download(headers_pc, randomproxy, cookie_dict, item, article_urllist)
with open('D:/Intern_Project/7_nhs/medicines_article链接列表.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(article_urllist, indent=2, ensure_ascii=False))

NameError: name 'randomproxy' is not defined

In [15]:
def medicines_articlehtml_download(headers_pc, randomproxy, cookie_dict, item):
    try:
        url = item['url']
        # title = item['title'].strip().replace('/', ' ').replace('?', ' ').replace('/"', ' ')   # To do：这里不能用title进行判断，title是重复的，使用url，结合正则表达式判断
        # title = item['url'].match(matchReg).strip().replace('/', ' ') 
        index = item['url'].find("https://www.nhs.uk/medicines/")
        title = item['url'][index + len("https://www.nhs.uk/medicines/"):].replace('/', '：').replace(':', '：')
        if os.path.exists(f"D:/Intern_Project/7_nhs/medicineslist_html/medicines_html/{title}.html"):
            pbar.update(1)    
            return None 
#         time.sleep(random.randint(0, 5))
        response = get_response(url, headers_pc, randomproxy, cookie_dict)
        
        with open(f"D:/Intern_Project/7_nhs/medicineslist_html/medicines_html/{title}.html", 'w+', encoding='utf-8') as f:
            f.write(response)
            
        pbar.update(1)
        # time.sleep(0.2)
    
    except Exception as e:
        print(e)
    return None

In [ ]:
headers_pc, randomproxy, cookie_dict = changeparams()

pbar = tqdm(total=len(article_urllist))

all_craw_tasks = []
results = []

# let matchReg = /(?<=medicines\/).*?(?=\/)/

with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for item in article_urllist:
        tmp_task = mt_pool.submit(medicines_articlehtml_download, headers_pc, randomproxy, cookie_dict, item)
        all_craw_tasks.append(tmp_task)

    # 监测下载任务进展
    task_monitor(all_craw_tasks, mt_pool, results)


 44%|██████████████████████████████████▋                                            | 557/1267 [00:49<39:23,  3.33s/it]


 54%|██████████████████████████████████████████▉                                    | 689/1267 [01:19<15:59,  1.66s/it]


 65%|███████████████████████████████████████████████████▏                           | 821/1267 [01:42<13:15,  1.78s/it]


 75%|███████████████████████████████████████████████████████████▍                   | 953/1267 [02:12<11:45,  2.25s/it]


 81%|███████████████████████████████████████████████████████████████▍              | 1030/1267 [02:32<10:59,  2.78s/it]


In [12]:
# 正式下载medicines_html
with open('D:/Intern_Project/7_nhs/medicines_article链接列表.json', 'r', encoding='utf-8') as f:
    article_urllist = json.load(f)
# print(len(article_urllist))
# headers_pc, randomproxy, cookie_dict = changeparams()

# for item in article_urllist:
#     medicines_articlehtml_download(headers_pc, randomproxy, cookie_dict, item)